# Content Based Recommender System

# Informasi Dataset



In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


In [2]:
data_movies = pd.read_csv("/kaggle/input/movie-recommendation-system/movies.csv")
data_ratings = pd.read_csv("/kaggle/input/movie-recommendation-system/ratings.csv")

In [3]:
data_movies.head(-10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62408,209135,Jane B. by Agnès V. (1988),Documentary|Fantasy
62409,209137,The Reward's Yours... The Man's Mine (1969),Western
62410,209139,Rimsky-Korsakov (1953),Drama
62411,209141,And They Lived Happily Ever After (1976),Comedy


In [4]:
data_ratings.head(-10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000080,162541,7193,5.0,1240950032
25000081,162541,7361,4.5,1240953484
25000082,162541,8610,2.5,1240950626
25000083,162541,8905,3.0,1240949310


In [5]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [6]:
data_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [7]:
data_movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
data_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [9]:
print(f"Jumlah jenis Movies di data Movies  : {data_movies['movieId'].nunique()}")
print(f"Jumlah jenis Movies di data Ratings : {data_ratings['movieId'].nunique()}")

Jumlah jenis Movies di data Movies  : 62423
Jumlah jenis Movies di data Ratings : 59047


In [10]:
arr_mov_most =  data_ratings['movieId'].value_counts()[:50].index
arr_count_most = data_ratings['movieId'].value_counts()[:50].values
arr_mov_most


Index([  356,   318,   296,   593,  2571,   260,   480,   527,   110,  2959,
         589,  1196,     1,  4993,    50,  1210,  1198,  2858,   858,  5952,
        7153,    47,   457,  1270,   780,   150,   608,    32,  2028,  2762,
        3578,   592,   588,   364,  4306,   380,   590, 58559,   377,  4226,
        1580,  1704, 79132,  1240,  1291,  1197,  1136,  1721,  1265,   344],
      dtype='int64', name='movieId')

In [11]:
arr_title_most = [data_movies.loc[data_movies['movieId']==x]['title'].values[0] for x in arr_mov_most]

In [12]:
df = pd.DataFrame({
    "movieId" : arr_mov_most, 
    "title" : arr_title_most, 
    "jumlah" : arr_count_most
})

df.head(5)

,movieId,title,jumlah
0,356,Forrest Gump (1994),81491
1,318,"Shawshank Redemption, The (1994)",81482
2,296,Pulp Fiction (1994),79672
3,593,"Silence of the Lambs, The (1991)",74127
4,2571,"Matrix, The (1999)",72674


In [13]:
all_data = pd.merge(data_ratings, data_movies, on="movieId", how="left")
all_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [14]:
all_data_fix =  all_data.drop_duplicates("movieId")

In [15]:
all_data_fix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59047 entries, 0 to 24970610
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     59047 non-null  int64  
 1   movieId    59047 non-null  int64  
 2   rating     59047 non-null  float64
 3   timestamp  59047 non-null  int64  
 4   title      59047 non-null  object 
 5   genres     59047 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 3.2+ MB


In [16]:
all_data_fix.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [17]:
all_data_fix['genres'] = all_data_fix['genres'].apply(lambda x: x.lower())

/tmp/ipykernel_30/955519271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_fix['genres'] = all_data_fix['genres'].apply(lambda x: x.lower())


In [18]:
all_data_fix.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),comedy|crime|drama|thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),drama
3,1,665,5.0,1147878820,Underground (1995),comedy|drama|war
4,1,899,3.5,1147868510,Singin' in the Rain (1952),comedy|musical|romance


In [19]:
all_data_fix.drop(['timestamp', 'userId'], axis=1, inplace=True)
all_data_fix.head()

/tmp/ipykernel_30/3492369663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_fix.drop(['timestamp', 'userId'], axis=1, inplace=True)


,movieId,rating,title,genres
0,296,5.0,Pulp Fiction (1994),comedy|crime|drama|thriller
1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),drama
2,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),drama
3,665,5.0,Underground (1995),comedy|drama|war
4,899,3.5,Singin' in the Rain (1952),comedy|musical|romance


In [20]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity

all_data_fix["text"] = all_data_fix["title"] + " " + all_data_fix["genres"].str.replace("|", " ")
all_data_fix.head()

/tmp/ipykernel_30/2164145096.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_fix["text"] = all_data_fix["title"] + " " + all_data_fix["genres"].str.replace("|", " ")


,movieId,rating,title,genres,text
0,296,5.0,Pulp Fiction (1994),comedy|crime|drama|thriller,Pulp Fiction (1994) comedy crime drama thriller
1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),drama,Three Colors: Red (Trois couleurs: Rouge) (199...
2,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),drama,Three Colors: Blue (Trois couleurs: Bleu) (199...
3,665,5.0,Underground (1995),comedy|drama|war,Underground (1995) comedy drama war
4,899,3.5,Singin' in the Rain (1952),comedy|musical|romance,Singin' in the Rain (1952) comedy musical romance


In [21]:
words_token = [str(x).split() for x in all_data_fix['text']]
words_token[:2]

[['Pulp', 'Fiction', '(1994)', 'comedy', 'crime', 'drama', 'thriller'],
 ['Three',
  'Colors:',
  'Red',
  '(Trois',
  'couleurs:',
  'Rouge)',
  '(1994)',
  'drama']]

In [22]:
model = FastText(words_token, vector_size=100, window=3, min_count=1, epochs=30)

In [23]:
def get_embedding(text):
    word = str(text).split()
    vector = [model.wv[x] for x in word if x in model.wv]
    if vector:
        return sum(vector) / len(vector)
    else:
        return np.zeros(vector.vector_size)

In [24]:
all_data_fix["embedding"] = all_data_fix['text'].apply(get_embedding)
all_data_fix.head()

/tmp/ipykernel_30/3143921955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_fix["embedding"] = all_data_fix['text'].apply(get_embedding)


,movieId,rating,title,genres,text,embedding
0,296,5.0,Pulp Fiction (1994),comedy|crime|drama|thriller,Pulp Fiction (1994) comedy crime drama thriller,"[0.043347266, 0.17964801, -0.25673383, -0.0696..."
1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),drama,Three Colors: Red (Trois couleurs: Rouge) (199...,"[-0.27154952, -0.42080423, 0.14147693, -0.0206..."
2,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),drama,Three Colors: Blue (Trois couleurs: Bleu) (199...,"[-0.053206384, -0.33145478, 0.10638151, -0.093..."
3,665,5.0,Underground (1995),comedy|drama|war,Underground (1995) comedy drama war,"[0.031042123, 0.41182128, 0.31346163, -0.44129..."
4,899,3.5,Singin' in the Rain (1952),comedy|musical|romance,Singin' in the Rain (1952) comedy musical romance,"[-0.36727777, 0.5156449, 0.74454975, -0.886857..."


In [25]:
all_data_fix["embedding"][0]

array([ 0.04334727,  0.17964801, -0.25673383, -0.06963165,  1.1942718 ,
       -0.2601125 ,  0.73009163,  0.5090265 , -1.3391768 , -0.35768488,
        0.7722066 ,  0.3603522 ,  0.0170944 ,  1.0247982 , -0.1610476 ,
        0.46597964,  0.8207146 ,  1.0586189 , -0.58385754, -0.44571108,
       -0.21361975, -1.1943098 ,  0.446043  ,  0.7312881 ,  1.3621019 ,
       -0.9907898 , -0.08766699, -0.62082756,  0.3045223 ,  0.44941062,
        0.6777895 ,  0.21568188,  0.89041907, -2.3957596 , -0.7808007 ,
        0.66565925, -0.5344654 , -0.4660725 ,  0.9444712 , -0.10575165,
        0.25160623, -1.0172606 ,  0.2577171 , -0.16431697, -0.37803072,
        0.09551191, -0.57903945, -0.0995746 , -0.20879778, -0.02963845,
       -0.8568758 , -1.3097125 ,  0.25359055, -0.62910354,  0.19219173,
       -0.41615555, -0.5323594 ,  0.05259182,  0.01929238,  0.26993248,
        0.8939613 ,  0.09742486,  0.49456862,  0.4465509 , -0.3202779 ,
        1.4246747 , -0.06564675,  0.660682  , -0.01818897,  0.91

In [26]:
embedd = np.vstack(all_data_fix["embedding"].to_numpy())
similarity_matrix = cosine_similarity(embedd)

In [27]:
similarity_matrix[:2]

array([[1.0000001 , 0.69218075, 0.72747636, ..., 0.3232444 , 0.7334402 ,
        0.35955623],
       [0.69218075, 0.99999994, 0.9563742 , ..., 0.44629997, 0.7578991 ,
        0.42989814]], dtype=float32)

In [53]:
all_data_fix = all_data_fix.reset_index(drop=True)
all_data_fix.head(-1)

,movieId,rating,title,genres,text,embedding
0,296,5.0,Pulp Fiction (1994),comedy|crime|drama|thriller,Pulp Fiction (1994) comedy crime drama thriller,"[0.043347266, 0.17964801, -0.25673383, -0.0696..."
1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),drama,Three Colors: Red (Trois couleurs: Rouge) (199...,"[-0.27154952, -0.42080423, 0.14147693, -0.0206..."
2,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),drama,Three Colors: Blue (Trois couleurs: Bleu) (199...,"[-0.053206384, -0.33145478, 0.10638151, -0.093..."
3,665,5.0,Underground (1995),comedy|drama|war,Underground (1995) comedy drama war,"[0.031042123, 0.41182128, 0.31346163, -0.44129..."
4,899,3.5,Singin' in the Rain (1952),comedy|musical|romance,Singin' in the Rain (1952) comedy musical romance,"[-0.36727777, 0.5156449, 0.74454975, -0.886857..."
...,...,...,...,...,...,...
59041,200190,2.0,Bombay Dreams (2006),comedy|drama,Bombay Dreams (2006) comedy drama,"[0.48969346, 0.002598411, -0.017264938, 0.0251..."
59042,200192,2.0,Den frusna leoparden (1986),(no genres listed),Den frusna leoparden (1986) (no genres listed),"[0.19133446, 0.58161646, 0.4868777, 1.0022978,..."
59043,200194,2.0,Tough Luck (2004),action|adventure|thriller,Tough Luck (2004) action adventure thriller,"[0.015396555, 0.38291952, -0.2407608, -0.22670..."
59044,139970,3.5,I Don't Speak English (1995),comedy,I Don't Speak English (1995) comedy,"[0.8599434, -1.7560105, -1.0747143, 0.9450268,..."


In [59]:
movie_id = 200726
movie_data = all_data_fix[all_data_fix['movieId']==movie_id]["movieId"]
film_index = movie_data.values[0]
similar_movies = similarity_matrix[movie_data.index[0]].argsort()[::-1][1:10]
print(f"Film : {all_data_fix.loc[all_data_fix['movieId']==film_index][['movieId', 'title', 'genres']]}")

print("Rekomendasi:")
pd.DataFrame({
    "movieId" : all_data_fix.iloc[similar_movies]["movieId"],
    "title" : all_data_fix.iloc[similar_movies]["title"],
    "genres" : all_data_fix.iloc[similar_movies]["genres"],
}).head(-1)


Film :        movieId                 title          genres
59045   200726  The Graduates (1995)  children|drama
Rekomendasi:


,movieId,title,genres
40890,146724,The Challengers (1990),children|drama
14517,107865,"Glass-blower's Children, The (Glasblåsarns bar...",children|drama
52775,88801,"Yearling, The (1994)",adventure|children|drama
40530,162330,The Children (1990),drama
1284,531,"Secret Garden, The (1993)",children|drama
205,5103,"Sandlot, The (1993)",children|comedy|drama
1266,343,"Baby-Sitters Club, The (1995)",children
5243,80,"White Balloon, The (Badkonake sefid) (1995)",children|drama
